In [6]:
# python program to distinguish tech articles from non tech articles using naive bayes and k-means clustering
import requests
from collections import defaultdict
from string import punctuation
from heapq import nlargest
from bs4 import BeautifulSoup
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.corpus import stopwords
import urllib2

import urllib
from cookielib import CookieJar
cj = CookieJar()
opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cj))



from math import log

hdr = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
      'Accept': 'text/html, application/xhtml+xml,application/xml;q=0.9,*/*q=0.8',
       'Accept-Charset':'ISO-8859-1;utf-8,q=0.7,*;q=0.3',
       'Accept-Encoding':'none',
       'Accept-Language':'en-US,en;q=0.8',
       'Connection':'keep-alive'
      }



def getWashPosText(url, token):
    try:
        #page_text = urllib.request.urlopen(url).read().decode('utf-8')
        
        #request = urllib2.Request(url)
        #page_text = urllib2.urlopen(request).read()
    
    #response = urllib.request.urlopen(url).read().decode('utf-8')
    
        req = urllib2.Request(url, headers=hdr)
        response = urllib2.urlopen(req)
        page_text = response.read()
    
    
    except e:
        print 'Error'
        return (None, None)
    soup = BeautifulSoup(page_text)
    
    if soup is None:
        return (None, None)
    
    text = ''
    if soup.find_all(token) is not None:
        text = ' '.join(map(lambda p : p.text, soup.find_all(token)))
        
        soup2 = BeautifulSoup(text)
        
        if soup2.find_all('p')!=[]:
            text = ' '.join(map(lambda p : p.text , soup2.find_all('p')))
            
    return text, soup.title.text


def getNYTText(url,token):
    #response = requests.get(url)
    # THis is an alternative way to get the contents of a URL
    
   
    p = opener.open("http://www.nytimes.com/2016/12/22/technology/apple-nokia-patent-smartphone.html?src=me")
    page_text= p.read()
    
    
    #req = urllib2.Request(url, headers=hdr)
    #response = urllib2.urlopen(req)
    #page_text = response.read()
    soup = BeautifulSoup(page_text)
    
    #page = str(soup)
    title = soup.find('title').text
    mydivs = soup.findAll("p", {"class":"story-body-text story-content"})
    text = ''.join(map(lambda p:p.text, mydivs))
    return text, title


def scrapeSource(url, magicFrag= '2016' , scraperFunction = 'getNYText' , token = None):
    urlBodies={}
    #request = urllib.request.Request(url)
    #response = urllib.request.urlopen(request)
    
    req = urllib2.Request(url , headers=hdr)
    response = urllib2.urlopen(req)
    page_text = response.read()
    soup = BeautifulSoup(page_text)
    
    
    #response = urllib.request.urlopen(url).read().decode('utf-8')
    
    #soup = BeautifulSoup(response)
    
    numErrors = 0
    for a in soup.find_all('a'):
        try:
            url = a['href']
            if((url not in urlBodies) and ((magicFrag is not None and magicFrag in url) or magicFrag is None)):
                body = scraperFunction(url, token)
            
                if body and len(body)>0:
                    urlBodies[url]=body
                print url
        except:
            numErrors+=1
    return urlBodies


        

        
    

class FrequencySummarizer:
    def __init__ (self, min_cut=0.1 , max_cut = 0.9):
        self._min_cut = min_cut
        self._max_cut = max_cut
        self._stopwords = set(stopwords.words('english') + list(punctuation) + [u"'s",'"'])
        
        
    def _computeFrequencies(self, word_sent , customStopWords= None):
        freq = defaultdict(int)
        if customStopWords is None:
            stopwords = self._stopwords
        else:
            stopwords = set(customStopWords).union(self._stopwords)
            
        for sentence in word_sent:
            for word in sentence:
                if word not in stopwords:
                    freq[word]+=1
                    
        m = float(max(freq.values()))
        for w in freq.keys():
            freq[w] = freq[w]/m
            if freq[w] >= self._max_cut or freq[w] <= self._min_cut:
                del freq[w]
            
        return freq
        
        
    def extractFeatures(self, article , n, customStopWords=None):
        text = article[0]
        title = article[1]
        
        
        sentences = sent_tokenize(text)
        word_sent = [word_tokenize(s.lower()) for s in sentences]
        self._freq = self._computeFrequencies(word_sent, customStopWords)
        
        if n < 0:
            return nlargest(len(self._freq.keys()) , self._freq , key = self._freq.get)
        else:
            return nlargest(n, self._freq , key = self._freq.get)
        
        
    def extractRawFrequencies(self, article):
        text = article[0]
        title = article[1]
        sentences = sent_tokenize(text)
        words_sent = [word_tokenize(s.lower()) for s in sentences]
        freq = defaultdict()
        for s in words_sent:
            for word in s:
                print word
                if word not in self._stopwords:
                    freq[word] += 1
        return freq
    
    def summarize(self, article , n):
        text = article[0]
        title = article[1]
        
        sentences = sent_tokenize(text)
        words_sent = [word_tokenize(s.lower()) for s in sentences]
        self._freq = self._computeFrequencies(words_sent)
        
        ranking = defaultdict()
        for i, sentence in enumerate(word_sent):
            for word in sentence:
                if word in self._freq:
                    ranking[i]+= self._freq[word]
                    
        sentences_index = nlargest(n, ranking, key = ranking.get)
        
        return [sentences[j] for j in sentences_index]
    
    
        
            
        
        

In [7]:
urlWashingtonPostNonTech = 'https://www.washingtonpost.com/sports/?utm_term=.a2b52a2aa5af'
urlNewYorkTimesNonTech ='http://www.nytimes.com/section/sports'

urlWashingtonPostTech = 'https://www.washingtonpost.com/business/technology/?nid=menu_nav_tech&utm_term=.edbde0c94842'
urlNewYorkTimesTech = 'http://www.nytimes.com/pages/technology/index.html?module=SectionsNav&action=click&version=BrowseTree&region=TopBar&contentCollection=Tech&pgtype=sectionfront'



washingtonPostTechArticles = scrapeSource(urlWashingtonPostTech , "2016" , getWashPosText , 'article')
washingtonPostNonTechArticles = scrapeSource(urlWashingtonPostNonTech , "2016" , getWashPosText , 'article')

newYorkTimesTechArticles = scrapeSource(urlNewYorkTimesTech , '2016' , getNYText, None)
newYorkTimesNonTechArticles = scrapeSource(urlNewYorkTimesNonTech , '2016' , getNYText, None)




https://www.washingtonpost.com/world/europe/russia-avoids-suggesting-terrorism-caused-plane-crash-that-killed-92-en-route-to-syria/2016/12/26/3c8f0198-cb78-11e6-85cd-e66532e35a44_story.html
https://www.washingtonpost.com/news/sports/wp/2016/12/26/is-everything-shaping-up-for-a-cowboys-patriots-super-bowl-the-nfls-tv-ratings-dream/?utm_term=.6c286c22ffc0
https://www.washingtonpost.com/local/obituaries/comedian-ricky-harris-from-everybody-hates-chris-dies/2016/12/27/ad43c80e-cbfc-11e6-85cd-e66532e35a44_story.html
https://www.washingtonpost.com/news/arts-and-entertainment/wp/2016/12/25/ae-abruptly-cancels-kkk-docu-series-before-it-airs/
https://www.washingtonpost.com/national/health-science/transition-advisers-tweets-add-intrigue-to-trump-aides-sudden-resignation/2016/12/25/a5aa0018-cae4-11e6-a87f-b917067331bb_story.html
https://www.washingtonpost.com/news/early-lead/wp/2016/12/26/commissioner-nba-boycott-of-trumps-white-house-would-be-a-lost-opportunity/
https://www.washingtonpost.com/ne

NameError: name 'getNYText' is not defined

In [66]:
print newYorkTimesNonTechArticles

{'http://www.nytimes.com/2016/12/23/sports/jets-coach-todd-bowles-is-hospitalized.html': (None, None), 'http://www.nytimes.com/2016/12/22/sports/tennis-player-petra-kvitova-could-play-again-after-hand-surgery.html': (None, None), 'http://www.nytimes.com/2016/12/22/sports/lebron-james-2016-nba-finals.html': (None, None), 'http://www.nytimes.com/2016/12/22/arts/design/muhammad-ali-drawn-painted-and-photographed.html': (None, None), 'http://www.nytimes.com/2016/12/23/sports/baseball/edwin-encarnacion-cleveland-indians.html': (None, None), 'http://www.nytimes.com/2016/12/15/sports/for-swimmers-with-ice-water-in-their-veins-an-event-to-match.html': (None, None), 'http://www.nytimes.com/2016/11/30/sports/football/terence-newman-minnesota-vikings-cornerback.html': (None, None), 'http://www.nytimes.com/2016/12/22/sports/football/fantasy-football-players-to-start-in-week-16.html': (None, None), 'http://www.nytimes.com/2016/12/23/sports/football/nfl-schedule-predictions-point-spread.html': (None

In [49]:
#print getNYTText('http://www.nytimes.com/2016/12/22/technology/apple-nokia-patent-smartphone.html?src=me', None)



import urllib2
import urllib
from cookielib import CookieJar

cj = CookieJar()
opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cj))
p = opener.open("http://www.nytimes.com/2016/12/22/technology/apple-nokia-patent-smartphone.html?src=me")

print p.read()

<!DOCTYPE html>
<!--[if (gt IE 9)|!(IE)]> <!--> <html lang="en" class="no-js section-technology format-medium tone-news app-article page-theme-standard has-top-ad type-size-small has-large-lede" itemid="http://www.nytimes.com/2016/12/22/technology/apple-nokia-patent-smartphone.html" itemtype="http://schema.org/NewsArticle"  itemscope xmlns:og="http://opengraphprotocol.org/schema/"> <!--<![endif]-->
<!--[if IE 9]> <html lang="en" class="no-js ie9 lt-ie10 section-technology format-medium tone-news app-article page-theme-standard has-top-ad type-size-small has-large-lede" xmlns:og="http://opengraphprotocol.org/schema/"> <![endif]-->
<!--[if IE 8]> <html lang="en" class="no-js ie8 lt-ie10 lt-ie9 section-technology format-medium tone-news app-article page-theme-standard has-top-ad type-size-small has-large-lede" xmlns:og="http://opengraphprotocol.org/schema/"> <![endif]-->
<!--[if (lt IE 8)]> <html lang="en" class="no-js lt-ie10 lt-ie9 lt-ie8 section-technology format-medium tone-news app-a

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans




In [76]:
articleSummaries = {}
for techUrlDictionary in [washingtonPostTechArticles]:
    for articleUrl in techUrlDictionary:
        if techUrlDictionary[articleUrl][0] is not None:
            if len(techUrlDictionary[articleUrl][0]) > 0:
                fs = FrequencySummarizer()
                summary = fs.extractFeatures(techUrlDictionary[articleUrl],25)
                articleSummaries[articleUrl] = {'feature-vector': summary,
                                               'label': 'Tech'}
                
for nontechUrlDictionary in [washingtonPostNonTechArticles]:
    for articleUrl in nontechUrlDictionary:
        if nontechUrlDictionary[articleUrl][0] is not None:
            if len(nontechUrlDictionary[articleUrl][0]) > 0:
                fs = FrequencySummarizer()
                summary = fs.extractFeatures(nontechUrlDictionary[articleUrl],25)
                articleSummaries[articleUrl] = {'feature-vector': summary,
                                               'label': 'Non-Tech'}


In [110]:
print articleSummaries

{'https://www.washingtonpost.com/news/dc-sports-bog/wp/2016/12/22/good-news-the-redskins-arent-as-bad-on-saturdays-as-they-are-on-mondays/': {'feature-vector': [u'saturday', u'games', u'night', u'washington', u'win', u'played', u'loss', u'home', u'chicago', u'christmas', u'bears', u'two', u'time', u'including', u'field', u'it\u2019s', u'nfc', u'lost', u'nfl', u'eve', u'fewer', u'since', u'fedex', u'days', u'one'], 'label': 'Non-Tech'}, 'https://www.washingtonpost.com/news/sports/wp/2016/12/23/jaguars-and-rams-coaching-jobs-arent-as-unattractive-as-you-might-think/': {'feature-vector': [u'nfl', u'said', u'jaguars', u'coach', u'one', u'office', u'coaching', u'executive', u'get', u'front', u'team', u'bortles', u'rams', u'l.a.', u'talent', u'made', u'think', u'overall', u'jacksonville', u'went', u'defense', u'place', u'draft', u'\u201d', u'since'], 'label': 'Non-Tech'}, 'https://www.washingtonpost.com/sports/redskins/jordan-reed-keeps-playing-through-injuries-maybe-someone-should-stop-him/

In [4]:
import urllib2
from bs4 import BeautifulSoup

def getDoxyDonkeyText(url , token):
        req = urllib2.Request(url, headers=hdr)
        response = urllib2.urlopen(req)
        page_text = response.read()
        
        soup = BeautifulSoup(page_text)
        
        title = soup.title.text
        mydivs = soup.find_all('div' , {'class': token})
        text = ' '.join(map(lambda p : p.text , mydivs))
        
        return text , title
    
testUrl = 'http://doxydonkey.blogspot.in'
testArticle = getDoxyDonkeyText(testUrl, 'post-body')

fs = FrequencySummarizer()
testArticleSummary = fs.extractFeatures(testArticle , 25)


        

C:\Users\Ashish Agrawal\Anaconda2\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file C:\Users\Ashish Agrawal\Anaconda2\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


NameError: name 'FrequencySummarizer' is not defined

In [92]:
print testArticleSummary

[u'``', u'security', u'use', u'india', u'mobile', u'payment', u'said', u'digital', u'don\u2019t', u'\u201d', u'wallet', u'say', u'banking', u'experts', u"''", u'details', u'risk', u'says', u'financial', u'passwords', u'card', u'online', u'\u2013', u'expert', u'percent']


In [93]:


similarities = {}
for articleUrl in articleSummaries:
    oneArticleSummary = articleSummaries[articleUrl]['feature-vector']
    similarities[articleUrl] = len(set(testArticleSummary).intersection(set(oneArticleSummary)))

labels = defaultdict(int)    
knn = nlargest(5, similarities, key=similarities.get)
for oneNeighbor in knn:
    labels[articleSummaries[oneNeighbor]['label']] += 1

nlargest(1,labels,key=labels.get)[0]




    
    

'Tech'

In [94]:
print testArticleSummary

[u'``', u'security', u'use', u'india', u'mobile', u'payment', u'said', u'digital', u'don\u2019t', u'\u201d', u'wallet', u'say', u'banking', u'experts', u"''", u'details', u'risk', u'says', u'financial', u'passwords', u'card', u'online', u'\u2013', u'expert', u'percent']


In [107]:
#naive bayes classification


cumulativeRawFrequencies = {'Tech':defaultdict(),'Non-Tech':defaultdict()}
trainingData = {'Tech':washingtonPostTechArticles, 'Non-Tech':washingtonPostNonTechArticles}

for label in trainingData:
    for articleUrl in trainingData[label]:
        if len(trainingData[label][articleUrl][0]) > 0 :
            fs = FrequencySummarizer()
            rawFrequencies = fs.extractRawFrequencies(trainingData[label][articleUrl])
            for word in rawFrequencies:
                cumulativeFrequencies[label][word]+= rawFrequencies[word]
                
                


kirk


KeyError: u'kirk'

In [105]:
washingtonPostTechArticles

{'http://www.washingtonpost.com/local/nine-whole-foods-managers-fired-over-alleged-stolen-bonuses-sue-for-defamation/2016/12/23/be755bee-c885-11e6-bf4b-2c064d32a4bf_story.html': (u'Whole Foods stores are packed with customers ahead of Christmas, but it has also been an eventful legal season for the chain in the Washington region, where nine managers were fired for allegedly stealing bonuses and at least two lawsuits were filed against the company in recent weeks. The company announced last week that nine D.C.-area store managers were fired after being accused of tampering with store bonuses paid to employees. The managers, at stores in Maryland, Virginia and D.C., were reportedly gaming the chain\u2019s \u201cgainsharing\u201d program, which awards bonuses to employees whose departments come in under budget.  [Whole Foods accused of cheating workers out of bonuses in class-action lawsuit]  Then, on Tuesday, one current and one former employee of the P Street NW Whole Foods in the Distr